In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"

In [2]:
import torch
from cephdataset import *

In [3]:
from train.pretrain import *
from train.pretrain import get_args
from mltool.universal_model_util import get_model_para_detail
import numpy as np

In [4]:
ckpt_path = "checkpoints/WeathBench64x128/CombM_UVTPHSC2p2uvth-AFNONet/ts_2_pretrain-physics_small_per_6_step/02_02_14_42_58217-seed_42"
args=get_args(os.path.join(ckpt_path,"config.json"))

In [5]:
args.port = 1

In [6]:
args.use_wandb=0
args.gpu = args.local_rank = gpu  = local_rank = 0
##### parse args: dataset_kargs / model_kargs / train_kargs  ###########
args= parse_default_args(args)
SAVE_PATH = get_ckpt_path(args)
SAVE_PATH = "debug"
args.SAVE_PATH = str(SAVE_PATH)
#args.pretrain_weight = os.path.join(args.SAVE_PATH,'pretrain_latest.pt')
########## inital log ###################
logsys = create_logsys(args,False)
args.distributed = False

if args.distributed:
    if args.dist_url == "env://" and args.rank == -1:
        args.rank = int(os.environ["RANK"])
    if args.multiprocessing_distributed:
        # For multiprocessing distributed training, rank needs to be the
        # global rank among all the processes
        args.rank = args.rank * ngpus_per_node + local_rank
    logsys.info(f"start init_process_group,backend={args.dist_backend}, init_method={args.dist_url},world_size={args.world_size}, rank={args.rank}")
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,world_size=args.world_size, rank=args.rank)

model           = build_model(args)
#param_groups    = timm.optim.optim_factory.add_weight_decay(model, args.weight_decay)
optimizer,lr_scheduler,criterion = build_optimizer(args,model)
loss_scaler     = torch.cuda.amp.GradScaler(enabled=True)
logsys.info(f'use lr_scheduler:{lr_scheduler}')

2023-02-06 13:23:22,292 model args: img_size= (64, 128)
2023-02-06 13:23:22,293 model args: patch_size= 1


log at debug
wandb id: None
wandb is off, the recorder list is  ['tensorboard'], we pass wandb
load UVTPHSC2p model from checkpoints/WeathBench64x128/UVTPHSC2p-AFNONet/ts_2_pretrain-physics_small_per_6_step/01_28_17_11_64255-seed_42/backbone.best.pt


2023-02-06 13:23:23,891 use model ==> CombM_UVTPHSC2p2uvth
2023-02-06 13:23:23,893 Rank: 0, Local_rank: 0 | Number of Parameters: 155053374, Number of Buffers: 0, Size of Model: 591.4817 MB



load UVTPHSCp2uvth model from 


2023-02-06 13:23:25,698 use lr_scheduler:<timm.scheduler.cosine_lr.CosineLRScheduler object at 0x7fc25fb6af40>


notice we are in property_pick mode, be careful. Current dataset is WeathBench64x128


In [36]:
args.mode = 'fourcast'

In [37]:
pretrain_path = os.path.join(ckpt_path,"backbone.best.pt")
args.pretrain_weight = pretrain_path

In [38]:
logsys.info(f"loading weight from {args.pretrain_weight}")
start_epoch, start_step, min_loss = load_model(model.module if args.distributed else model, optimizer, lr_scheduler, loss_scaler, path=args.pretrain_weight, 
                    only_model= (args.mode=='fourcast') or (args.mode=='finetune' and not args.continue_train) ,loc = 'cuda:{}'.format(args.gpu))
if args.more_epoch_train:
    assert args.pretrain_weight
    print(f"detect more epoch training, we will do a copy processing for {args.pretrain_weight}")
    os.system(f'cp {args.pretrain_weight} {args.pretrain_weight}-epoch{start_epoch}')
logsys.info("done!")

2023-02-06 13:45:42,600 loading weight from checkpoints/WeathBench64x128/CombM_UVTPHSC2p2uvth-AFNONet/ts_2_pretrain-physics_small_per_6_step/02_02_14_42_58217-seed_42/backbone.best.pt


loading model from checkpoints/WeathBench64x128/CombM_UVTPHSC2p2uvth-AFNONet/ts_2_pretrain-physics_small_per_6_step/02_02_14_42_58217-seed_42/backbone.best.pt...........


2023-02-06 13:45:43,058 done!


loading model weight success...........
loading model success...........


#### test

In [39]:
args.valid_batch_size = 4
args.dataset_kargs['use_offline_data']=0
test_dataset,  test_dataloader = get_test_dataset(args,test_dataset_tensor=None,test_record_load=None)

use dataset in datasets/weatherbench64x128


In [40]:
data_loader = test_dataloader
random_repeat = 0
snap_index=None
model.eval()
logsys.eval()
status     = 'test'
gpu        = dist.get_rank() if hasattr(model,'module') else 0
Fethcher   = Datafetcher
prefetcher = Fethcher(data_loader,next(model.parameters()).device)
batches = len(data_loader)
inter_b    = logsys.create_progress_bar(batches,unit=' img',unit_scale=data_loader.batch_size)
device = next(model.parameters()).device
data_cost = train_cost = rest_cost = 0 
now = time.time()
model.clim = torch.Tensor(data_loader.dataset.clim_tensor).to(device)
fourcastresult={}
save_prediction_first_step = None#torch.zeros_like(data_loader.dataset.data)
save_prediction_final_step = None#torch.zeros_like(data_loader.dataset.data)
# = 100
intervel = batches//logsys.log_trace_times + 1

with torch.no_grad():
    inter_b.lwrite("load everything, start_validating......", end="\r")
    while inter_b.update_step():
        #if inter_b.now>10:break
        data_cost  += time.time() - now;now = time.time()
        step        = inter_b.now
        idxes,batch = prefetcher.next()
        batch       = make_data_regular(batch,half_model)
        break
        out         = model(batch[0]) #(B,T,P,W,H) (B,T,4) (B,T,2,W,H)
        if torch.isnan(out).any():
            break

In [41]:
dataset = data_loader.dataset
time_step_1_mode = False

In [107]:
batch[0][0][-1].shape

torch.Size([64, 128])

In [30]:
do_error_propagration_monitor = None

In [44]:
model.use_amp

True

In [43]:
with torch.no_grad():
    accu_series=[]
    rmse_series=[]
    rmse_maps = []
    hmse_series=[]
    extra_info = {}
    time_step_1_mode=False
    batch_variance_line_pred = [] 
    batch_variance_line_true = []
    # we will also record the variance for each slot, assume the input is a time series of data
    # [ (B, P, W, H) -> (B, P, W, H) -> .... -> (B, P, W, H) ,(B, P, W, H)]
    # we would record the variance of each batch on the location (W,H)
    error_information = None
    clim = model.clim
    
    start = batch[0:model.history_length] # start must be a list    
    
    snap_line = []
    if (snap_index is not None) and (0 not in [len(t) for t in snap_index]):  
        for i,tensor in enumerate(start):
            # each tensor is like (B, 70, 32, 64) or (B, P, Z, W, H)
            snap_line.append([len(snap_line), get_tensor_value(tensor,snap_index, time=model.history_length),'input'])
    
    # approx_epsilon_lists = []
    # last_pred = last_target = None
    i = model.history_length
    while i<len(batch):#for i in range(model.history_length,len(batch), model.pred_len):# i now is the target index
        
        if do_error_propagration_monitor and i < do_error_propagration_monitor:
            # in this mode, we will concat all batch to accelarate computing. so far, only support batch=[X_{t},X_{t+1},X_{t+2}]
            monitor_batch = torch.stack(batch[:do_error_propagration_monitor],1) #[X_{t},X_{t+1},...,X_{t+2}]-> (B,T,P,W,H)
            ltmv_preds, ltmv_trues,error_information = once_forward_error_evaluation(model,monitor_batch)
            i = do_error_propagration_monitor-1
            start = [ltmv_preds[:,-1]]
            time_list  = range(1,do_error_propagration_monitor)
            ltmv_trues = ltmv_trues.transpose(0,1) #(B,T,P,W,H) -> (T,B,P,W,H)
            ltmv_preds = ltmv_preds.transpose(0,1) #(B,T,P,W,H) -> (T,B,P,W,H)
        else:
            end = batch[i:i+model.pred_len]
            end = end[0] if len(end) == 1 else end
            print(start[-1][0].mean(0))
            ltmv_pred, target, extra_loss, extra_info_from_model_list, start = once_forward(model,i,start,end,dataset,time_step_1_mode)
            print(ltmv_pred[0].mean(0))
            raise

tensor([[36.2699, 36.4158, 36.5667,  ..., 35.8370, 35.9752, 36.1191],
        [35.2991, 35.5539, 35.8285,  ..., 34.2741, 34.6441, 34.9707],
        [31.1176, 32.1553, 33.0155,  ..., 27.9668, 28.8431, 29.9111],
        ...,
        [-0.4025, -0.3985, -0.3531,  ..., -0.4262, -0.3990, -0.4274],
        [-0.3562, -0.3667, -0.3583,  ..., -0.3837, -0.3724, -0.3724],
        [-0.4651, -0.4711, -0.4768,  ..., -0.4584, -0.4581, -0.4616]],
       device='cuda:0')
tensor([[-0.4600, -0.4460, -0.4332,  ..., -0.4970, -0.4865, -0.4861],
        [-0.6113, -0.6046, -0.5650,  ..., -0.6573, -0.6401, -0.6298],
        [-0.6705, -0.6452, -0.6110,  ..., -0.6696, -0.6740, -0.6811],
        ...,
        [-0.4954, -0.4787, -0.4242,  ..., -0.4608, -0.4616, -0.4801],
        [-0.3850, -0.3872, -0.3754,  ..., -0.4157, -0.3994, -0.3904],
        [-0.4422, -0.4432, -0.4531,  ..., -0.4639, -0.4601, -0.4527]],
       device='cuda:0')


RuntimeError: No active exception to reraise

In [ ]:

#uvth = model.UVTPHSCp2uvth(torch.cat([UVTPHSC,p],1))
#uvtph= torch.cat([uvth[:,:42], p, uvth[:,42:]],1)

In [65]:
with torch.no_grad():
    UVTPHSC = batch[0]
    with torch.cuda.amp.autocast(enabled=model.use_amp):
        p    = model.UVTPHSC2p(UVTPHSC)
        x    = torch.cat([UVTPHSC,p],1)
        print(x.isnan().any())
        
        shape = x.shape
        #print(x.shape)
        # argue the w resolution.
        pad = model.UVTPHSCp2uvth.backbone.get_w_resolution_pad(shape)
        if pad is not None:x = F.pad(x.flatten(0,1),(0,0,pad,pad),mode='replicate').reshape(*shape[:-2],-1,shape[-1])
        #print(x.shape)
        B = x.shape[0]
        ot_shape = x.shape[2:]
        x = x.reshape(B,-1,*model.UVTPHSCp2uvth.backbone.img_size)# (B, p, z, h, w) or (B, p, h, w)
        #timer.restart(level=0)
        #print(torch.std_mean(x))
        print(x.isnan().any())
        
        #x = model.UVTPHSCp2uvth.backbone.forward_features(x);#print(torch.std_mean(x))
        x = model.UVTPHSCp2uvth.backbone.patch_embed(x)
        print(x.isnan().any())
        x += model.UVTPHSCp2uvth.backbone.pos_embed
        print(x.isnan().any())
        x = model.UVTPHSCp2uvth.backbone.pos_drop(x)
        print(x.isnan().any())
        #print(torch.std_mean(x))
        print("==================")
        if not model.UVTPHSCp2uvth.backbone.checkpoint_activations:
            for num,blk in enumerate(model.UVTPHSCp2uvth.backbone.blocks):
                x = blk(x);#print(torch.std_mean(x))
                print(x.isnan().any())
                if num==2:break
        else:
            x = checkpoint_sequential(model.UVTPHSCp2uvth.backbone.blocks, 4, x)
#         print("==================")
#         x = model.UVTPHSCp2uvth.backbone.norm(x).transpose(1, 2);
#         print(x.isnan().any())
#         x = torch.reshape(x, [-1, model.UVTPHSCp2uvth.backbone.embed_dim, *model.UVTPHSCp2uvth.backbone.final_shape])
#         print(x.isnan().any())
        #timer.record('forward_features',level=0)
#         x = model.UVTPHSCp2uvth.backbone.final_dropout(x)
        
        
#         #timer.record('final_dropout',level=0)
#         x = model.UVTPHSCp2uvth.backbone.pre_logits(x);#print(torch.std_mean(x))
        
#         print(x.isnan().any())
#         #timer.record('pre_logits',level=0)
#         x = model.UVTPHSCp2uvth.backbone.head(x)  # print(torch.std_mean(x))
        
#         print(x.isnan().any())
#         if model.UVTPHSCp2uvth.backbone.history_length >1:
#             x = x.flatten(1,2).transpose(1,-1)
#             x = model.UVTPHSCp2uvth.backbone.last_Linear_layer(x)
#             x = x.transpose(1,-1)
#             ot_shape=ot_shape[1:]
#         #timer.record('head',level=0)
#         x = x.reshape(B,-1,*ot_shape)
#         if pad is not None:
#             x = x[...,pad:-pad,:]

tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')


In [78]:
start_tensor = x.detach().clone()

In [84]:
model.UVTPHSCp2uvth.backbone.blocks[3].filter

AdaptiveFourierNeuralOperator(
  (relu): ReLU()
)

In [94]:
model.UVTPHSCp2uvth.backbone.blocks[3].norm1.weight.max()

tensor(41.9455, device='cuda:0', grad_fn=<MaxBackward1>)

In [92]:
model.UVTPHSCp2uvth.backbone.blocks[3].norm1.bias

Parameter containing:
tensor([-0.3199,  0.2006, -1.3664,  ..., -1.3705,  0.4756,  1.6072],
       device='cuda:0', requires_grad=True)

In [89]:
x.min()

tensor(-318.6756, device='cuda:0')

In [86]:
with torch.no_grad():
    with torch.cuda.amp.autocast(enabled=model.use_amp):
        x = start_tensor
        residual = x;print(x.isnan().any())
        #timer.restart(1)
        x = model.UVTPHSCp2uvth.backbone.blocks[3].norm1(x);print(x.isnan().any())
        #timer.record('norm1','forward_features',1)
#         x = model.UVTPHSCp2uvth.backbone.blocks[3].filter(x);print(x.isnan().any())
#         #timer.record('filter','forward_features',1)
#         if model.UVTPHSCp2uvth.backbone.blocks[3].double_skip:
#             x += residual
#             residual = x;
#         #timer.record('residual','forward_features',1)
#         x = model.UVTPHSCp2uvth.backbone.blocks[3].norm2(x);print(x.isnan().any())
#         #timer.record('norm2','forward_features',1)
#         x = model.UVTPHSCp2uvth.backbone.blocks[3].mlp(x);print(x.isnan().any())
#         #timer.record('mlp','forward_features',1)
#         x = model.UVTPHSCp2uvth.backbone.blocks[3].drop_path(x);print(x.isnan().any())
#         #timer.record('drop_path','forward_features',1)
#         x += residual
#         #timer.record('residual','forward_features',1)
#         #out = model.UVTPHSCp2uvth.backbone.blocks[3](x)

tensor(False, device='cuda:0')
tensor(False, device='cuda:0')


In [77]:
model.UVTPHSCp2uvth.backbone.blocks[3]

Block(
  (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
  (filter): AdaptiveFourierNeuralOperator(
    (relu): ReLU()
  )
  (drop_path): Identity()
  (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
  (mlp): Mlp(
    (fc1): Linear(in_features=1024, out_features=4096, bias=True)
    (act): GELU(approximate='none')
    (fc2): AdaptiveAvgPool1d(output_size=1024)
    (drop): Dropout(p=0.0, inplace=False)
  )
)

In [76]:
x.min()

tensor(-33.2812, device='cuda:0', dtype=torch.float16)

In [73]:
out

tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [na

In [55]:
model.UVTPHSCp2uvth.backbone.forward_features

<bound method AFNONet.forward_features of AFNONet(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(86, 1024, kernel_size=(1, 1), stride=(1, 1))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (filter): AdaptiveFourierNeuralOperator(
        (relu): ReLU()
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (fc2): AdaptiveAvgPool1d(output_size=1024)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (filter): AdaptiveFourierNeuralOperator(
        (relu): ReLU()
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(

In [46]:
CombM_UVTPHSC2p2uvth.

tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],

         ...,

         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, n

#### train

In [ ]:
#args.dataset_kargs['cross_sample']=args.cross_sample=0

In [16]:
args.dataset_kargs['batch_size']=args.batch_size = 1

In [17]:
args.dataset_kargs['cross_sample']=args.cross_sample=0

In [18]:
args.dataset_kargs['use_offline_data']=0

In [19]:
args.dataset_kargs['patch_range']=(3,5,5)

In [20]:
train_dataset, val_dataset, train_dataloader,val_dataloader = get_train_and_valid_dataset(args,
               train_dataset_tensor=None,train_record_load=None,
               valid_dataset_tensor=None,valid_record_load=None)
logsys.info(f"use dataset ==> {train_dataset.__class__.__name__}")
logsys.info(f"Start training for {args.epochs} epochs")
master_bar = logsys.create_master_bar(args.epochs)
accu_list = ['valid_loss']
metric_dict = logsys.initial_metric_dict(accu_list)

AssertionError: 

In [33]:
args.dataset_kargs['cross_sample']=args.cross_sample=1
train_dataset.cross_sample  =1
train_dataloader  = torch.utils.data.DataLoader(train_dataset,1)

In [34]:
train_dataset.cross_sample  =0
train_dataloader  = torch.utils.data.DataLoader(train_dataset,300)

In [ ]:
# val_dataset.cross_sample  =0 
# val_dataloader = torch.utils.data.DataLoader(val_dataset,256)

In [10]:
epoch = 0
start_step = 0
data_loader = train_dataloader
status = 'train'
if status == 'train':
    model.train()
    logsys.train()
elif status == 'valid':
    model.eval()
    logsys.eval()
else:
    raise NotImplementedError
accumulation_steps = model.accumulation_steps # should be 16 for finetune. but I think its ok.
half_model = next(model.parameters()).dtype == torch.float16

data_cost  = []
train_cost = []
rest_cost  = []
now = time.time()

Fethcher   = RandomSelectPatchFetcher if( status =='train' and \
                                          data_loader.dataset.use_offline_data and \
                                          data_loader.dataset.split=='train' and \
                                          'Patch' in data_loader.dataset.__class__.__name__) else Datafetcher
device     = next(model.parameters()).device
prefetcher = Fethcher(data_loader,device)
#raise
batches    = len(data_loader)

inter_b    = logsys.create_progress_bar(batches,unit=' img',unit_scale=data_loader.batch_size)
gpu        = dist.get_rank() if hasattr(model,'module') else 0

if start_step == 0:optimizer.zero_grad()
intervel = batches//100 + 1


total_diff,total_num  = torch.Tensor([0]).to(device), torch.Tensor([0]).to(device)
nan_count = 0
Nodeloss1 = Nodeloss2 = Nodeloss12 = -1

inter_b.lwrite(f"load everything, start_{status}ing......", end="\r")
preds = []
reals = []

In [52]:
train_dataset.cross_sample  =0
train_dataloader  = torch.utils.data.DataLoader(train_dataset,1)

In [46]:
val_dataset.cross_sample  = 1
val_dataloader  = torch.utils.data.DataLoader(val_dataset,1)

In [12]:
model.accumulation_steps = 1

In [53]:
import time

In [41]:
def run_one_iter(model, batch, criterion, status, gpu, dataset):
    iter_info_pool={}
    loss = 0
    diff = 0
    random_run_step = np.random.randint(1,len(batch)) if len(batch)>1 else 0
    time_step_1_mode=False
    if len(batch) == 1 and isinstance(batch[0],(list,tuple)) and len(batch[0])>1:
        batch = batch[0] # (Field, FieldDt)
        time_step_1_mode=True
    if model.history_length > len(batch):
        print(f"you want to use history={model.history_length}")
        print(f"but your input batch(timesteps) only has len(batch)={len(batch)}")
        raise
    pred_step = 0
    start = batch[0:model.history_length] # start must be a list
    ltmv_pred_record = []
    target_record = []
    for i in range(model.history_length,len(batch)):# i now is the target index
        ltmv_pred, target, extra_loss, extra_info_from_model_list, start = once_forward(model,i,start,batch[i],dataset,time_step_1_mode)
        if extra_loss !=0:
            iter_info_pool[f'{status}_extra_loss_gpu{gpu}_timestep{i}'] = extra_loss.item()
        for extra_info_from_model in extra_info_from_model_list:
            for name, value in extra_info_from_model.items():
                iter_info_pool[f'valid_on_{status}_{name}_timestep{i}'] = value
        
        ltmv_pred = dataset.do_normlize_data([ltmv_pred])[0]

        abs_loss = criterion(ltmv_pred,target)
        ltmv_pred_record.append(ltmv_pred)
        target_record.append(target)
        iter_info_pool[f'{status}_abs_loss_gpu{gpu}_timestep{i}'] =  abs_loss.item()
        pred_step+=1
        loss += abs_loss + extra_loss
        diff += abs_loss
        if model.random_time_step_train and i >= random_run_step:
            break
    # loss = loss/(len(batch) - 1)
    # diff = diff/(len(batch) - 1)
    loss = loss/pred_step
    diff = diff/pred_step
    return loss, diff, iter_info_pool,torch.cat(ltmv_pred_record),torch.cat(target_record)

In [55]:
epoch = 0
start_step = 0
data_loader = train_dataloader
status = 'valid'
if status == 'train':
    model.train()
    logsys.train()
elif status == 'valid':
    model.eval()
    logsys.eval()
else:
    raise NotImplementedError
accumulation_steps = model.accumulation_steps # should be 16 for finetune. but I think its ok.
half_model = next(model.parameters()).dtype == torch.float16

data_cost  = []
train_cost = []
rest_cost  = []
now = time.time()

Fethcher   = RandomSelectPatchFetcher if( status =='train' and \
                                          data_loader.dataset.use_offline_data and \
                                          data_loader.dataset.split=='train' and \
                                          'Patch' in data_loader.dataset.__class__.__name__) else Datafetcher
device     = next(model.parameters()).device
prefetcher = Fethcher(data_loader,device)
#raise
batches    = len(data_loader)

inter_b    = logsys.create_progress_bar(batches,unit=' img',unit_scale=data_loader.batch_size)
gpu        = dist.get_rank() if hasattr(model,'module') else 0

if start_step == 0:optimizer.zero_grad()
intervel = batches//100 + 1


total_diff,total_num  = torch.Tensor([0]).to(device), torch.Tensor([0]).to(device)
nan_count = 0
Nodeloss1 = Nodeloss2 = Nodeloss12 = -1

inter_b.lwrite(f"load everything, start_{status}ing......", end="\r")
preds = []
reals = []
while inter_b.update_step():
    #if inter_b.now>10:break
    step = inter_b.now
    batch = prefetcher.next()
    #print(batch[0].shape)
    #raise
    if step < start_step:continue
    #batch = data_loader.dataset.do_normlize_data(batch)

    batch = make_data_regular(batch,half_model)
    
    
    #if len(batch)==1:batch = batch[0] # for Field -> Field_Dt dataset
    data_cost.append(time.time() - now);now = time.time()
    if status == 'train':
        if hasattr(model,'set_step'):model.set_step(step=step,epoch=epoch)
        if hasattr(model,'module') and hasattr(model.module,'set_step'):model.module.set_step(step=step,epoch=epoch)
        if model.train_mode =='pretrain':
            time_truncate = max(min(epoch//3,data_loader.dataset.time_step),2)
            batch=batch[:model.history_length -1 + time_truncate]

        # the normal initial method will cause numerial explore by using timestep > 4 senenrio.
        if model.use_amp:
            with torch.cuda.amp.autocast():
                loss, abs_loss, iter_info_pool =run_one_iter(model, batch, criterion, 'train', gpu, data_loader.dataset)
        else:
            loss, abs_loss, iter_info_pool =run_one_iter(model, batch, criterion, 'train', gpu, data_loader.dataset)
        loss, nan_count, skip = nan_diagnose_weight(model,loss,nan_count,logsys)
        if skip:continue
        loss /= accumulation_steps

        if model.use_amp:
            loss_scaler.scale(loss).backward()
        else:
            loss.backward()

        if model.clip_grad:
            if model.use_amp:
                assert accumulation_steps == 1
                loss_scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), model.clip_grad)

        if (step+1) % accumulation_steps == 0:
            if model.use_amp:

                loss_scaler.step(optimizer)
                loss_scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()
    else:
        with torch.no_grad():
            #print([p.shape for t in batch for p in t])
            loss, abs_loss, iter_info_pool,pred,real =run_one_iter(model, batch, criterion, status, gpu, data_loader.dataset)
    iter_info_pool={}
    iter_info_pool[f'{status}_loss_gpu{gpu}']     =  loss.item()
    iter_info_pool[f'{status}_Nodeloss1_gpu{gpu}'] = Nodeloss1
    iter_info_pool[f'{status}_Nodeloss12_gpu{gpu}'] = Nodeloss12
    iter_info_pool[f'{status}_Nodeloss2_gpu{gpu}'] = Nodeloss2
    total_diff  += abs_loss.item()
    #total_num   += len(batch) - 1 #batch 
    total_num   += 1 

    train_cost.append(time.time() - now);now = time.time()
    time_step_now = len(batch)
    outstring=(f"epoch:{epoch:03d} iter:[{step:5d}]/[{len(data_loader)}] [TimeLeng]:{time_step_now:} GPU:[{gpu}] abs_loss:{abs_loss.item():.2f} loss:{loss.item():.2f} cost:[Date]:{np.mean(data_cost):.1e} [Train]:{np.mean(train_cost):.1e} ")
    #print(data_loader.dataset.record_load_tensor.mean().item())
    data_cost  = []
    train_cost = []
    rest_cost = []
    inter_b.lwrite(outstring, end="\r")
    preds.append(pred.detach().cpu())
    reals.append(real.detach().cpu())

KeyboardInterrupt: 

In [ ]:
# for epoch in mastera_bar:
#     if epoch < start_epoch:continue
#     if hasattr(model,'set_epoch'):model.set_epoch(epoch=epoch,epoch_total=args.epochs)
#     if hasattr(model,'module') and hasattr(model.module,'set_epoch'):model.module.set_epoch(epoch=epoch,epoch_total=args.epochs)
#     logsys.record('learning rate',optimizer.param_groups[0]['lr'],epoch, epoch_flag='epoch')
#     train_loss = run_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,logsys,'train')


NameError: name 'master_bar' is not defined

In [ ]:
train_dataset_tensor=None;
train_record_load=None;
valid_dataset_tensor=None;
valid_record_load=None

In [ ]:
# =======================> start training <==========================
print(f"entering {args.mode} training in {next(model.parameters()).device}")
now_best_path = SAVE_PATH / 'backbone.best.pt'
latest_ckpt_p = SAVE_PATH / 'pretrain_latest.pt'


train_dataset, val_dataset, train_dataloader,val_dataloader = get_train_and_valid_dataset(args,
               train_dataset_tensor=train_dataset_tensor,train_record_load=train_record_load,
               valid_dataset_tensor=valid_dataset_tensor,valid_record_load=valid_record_load)
logsys.info(f"use dataset ==> {train_dataset.__class__.__name__}")
logsys.info(f"Start training for {args.epochs} epochs")
metric_list = ['loss']
master_bar        = logsys.create_master_bar(args.epochs)
master_bar.set_multiply_graph(figsize=(9,3),engine=[['plot','plot']],labels=[metric_list])
for epoch in master_bar:
    if epoch < start_epoch:continue
    if hasattr(model,'set_epoch'):model.set_epoch(epoch=epoch,epoch_total=args.epochs)
    if hasattr(model,'module') and hasattr(model.module,'set_epoch'):model.module.set_epoch(epoch=epoch,epoch_total=args.epochs)
    logsys.record('learning rate',optimizer.param_groups[0]['lr'],epoch)
    train_loss = run_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,logsys,'train')
    if (not args.more_epoch_train) and (lr_scheduler is not None):lr_scheduler.step(epoch)
    #torch.cuda.empty_cache()
    #train_loss = single_step_evaluate(train_dataloader, model, criterion,epoch,logsys,status='train') if 'small' in args.train_set else -1
    val_loss   = run_one_epoch(epoch, start_step, model, criterion, val_dataloader, optimizer, loss_scaler,logsys,'valid')

    if (not args.distributed) or (args.rank == 0 and local_rank == 0) :
        logsys.info(f"Epoch {epoch} | Train loss: {train_loss:.6f}, Val loss: {val_loss:.6f}")
        logsys.record('train', train_loss, epoch)
        logsys.record('valid', val_loss, epoch)
        if use_wandb:wandb.log({"epoch":epoch,'train':train_loss,'valid':val_loss})
        if val_loss < min_loss:
            min_loss = val_loss
            if epoch > args.epochs//10:
                logsys.info(f"saving best model ....")
                save_model(model, path=now_best_path, only_model=True)
                logsys.info(f"done;")
            #if last_best_path is not None:os.system(f"rm {last_best_path}")
            #last_best_path= now_best_path
            logsys.info(f"The best accu is {val_loss}")
        logsys.record('best_loss', min_loss, epoch)
        update_experiment_info(experiment_hub_path,epoch,args)
        if epoch>args.save_warm_up:
            logsys.info(f"saving latest model ....")
            save_model(model, epoch+1, 0, optimizer, lr_scheduler, loss_scaler, min_loss, latest_ckpt_p)
            logsys.info(f"done ....")

if os.path.exists(now_best_path) and args.do_final_fourcast:
    logsys.info(f"we finish training, then start test on the best checkpoint {now_best_path}")
    start_epoch, start_step, min_loss = load_model(model.module if args.distributed else model, path=now_best_path, only_model=True)
    run_fourcast(args, model,logsys)
if use_wandb:wandb.finish()

In [ ]:
if local_rank == 0:
    print(f"Start training for {args.epochs} epochs")

master_bar        = logsys.create_master_bar(args.epochs)
last_best_path = None
for epoch in master_bar:
    if epoch < start_epoch:continue
    train_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,lr_scheduler, min_loss,logsys)
    lr_scheduler.step(epoch)
    #torch.cuda.empty_cache()
    train_loss = single_step_evaluate(train_dataloader, model, criterion,epoch,logsys)
    #train_loss = -1
    val_loss   = single_step_evaluate(val_dataloader, model, criterion,epoch,logsys)

    if rank == 0 and local_rank == 0:
        print(f"Epoch {epoch} | Train loss: {train_loss:.6f}, Val loss: {val_loss:.6f}")
        logsys.record('train', train_loss, epoch)
        logsys.record('valid', val_loss, epoch)
        if val_loss < min_loss:
            min_loss = val_loss
            print(f"saving best model ....")
            now_best_path = SAVE_PATH / f'backbone.best.pt'
            if epoch>args.save_warm_up:save_model(model, path=now_best_path, only_model=True)
            #if last_best_path is not None:os.system(f"rm {last_best_path}")
            #last_best_path= now_best_path
            print(f"done; the best accu is {val_loss}")
        logsys.record('best_loss', min_loss, epoch)
        

In [ ]:
if __name__ == '__main__':
    args = get_args()
    ngpus = ngpus_per_node = torch.cuda.device_count()
    args.world_size = -1
    args.dist_file  = None
    args.rank       = 0
    args.dist_backend = "nccl"
    args.multiprocessing_distributed = ngpus>1
    if not hasattr(args,'train_set'):args.train_set='large'
    ip = os.environ.get("MASTER_ADDR", "127.0.0.1")
    port = os.environ.get("MASTER_PORT", "54247")
    hosts = int(os.environ.get("WORLD_SIZE", "1"))  # number of nodes
    rank = int(os.environ.get("RANK", "0"))  # node id
    gpus = torch.cuda.device_count()  # gpus per node
    args.dist_url = f"tcp://{ip}:{port}"
    if args.world_size == -1 and "SLURM_NPROCS" in os.environ:
        args.world_size = int(os.environ["SLURM_NPROCS"])
        args.rank       = int(os.environ["SLURM_PROCID"])
        jobid           = os.environ["SLURM_JOBID"]

        hostfile        = "dist_url." + jobid  + ".txt"
        if args.dist_file is not None:
            args.dist_url = "file://{}.{}".format(os.path.realpath(args.dist_file), jobid)
        elif args.rank == 0:
            import socket
            ip = socket.gethostbyname(socket.gethostname())
            port = find_free_port()
            args.dist_url = "tcp://{}:{}".format(ip, port)
            #with open(hostfile, "w") as f:f.write(args.dist_url)
        else:
            import os
            import time
            while not os.path.exists(hostfile):
                time.sleep(1)
            with open(hostfile, "r") as f:
                args.dist_url = f.read()
        print("dist-url:{} at PROCID {} / {}".format(args.dist_url, args.rank, args.world_size))
    else:
        args.world_size = 1
    args.distributed = args.world_size > 1 or args.multiprocessing_distributed
    train_dataset_tensor=valid_dataset_tensor=None

    print("======== loading data ==========")
    if 'small' in args.train_set:
        if not args.fourcast:
            train_dataset_tensor = load_small_dataset_in_memory('train').share_memory_()
            valid_dataset_tensor = load_small_dataset_in_memory('valid').share_memory_()
        else:
            train_dataset_tensor = load_small_dataset_in_memory('test').share_memory_()
            valid_dataset_tensor = None
    else:
        if args.fourcast:
            train_dataset_tensor = load_test_dataset_in_memory(years=[2018],root="/nvme/zhangtianning/datasets/ERA5").share_memory_()
            valid_dataset_tensor = None
    print("=======done==========")
    print(train_dataset_tensor.shape)
    if args.multiprocessing_distributed:
        args.world_size = ngpus_per_node * args.world_size
        torch.multiprocessing.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args,train_dataset_tensor,valid_dataset_tensor))
    else:
        main_worker(0, ngpus_per_node, args,train_dataset_tensor,valid_dataset_tensor)